# CHANGE CURRENT WORKING DIRECTORY (COLAB)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "Temp"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  print('\nYour working directory was changed to ' + root_dir + project_folder)

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/Temp


# PARAMETERS

In [3]:
height = 256
width = 320
num_classes = 196
seed = 42
batch_size = 64
lr = 1e-4
num_epochs = 100
label_file = "names.csv"
train_file = "anno_train.csv"
test_file = "anno_test.csv"


# DATA PREPROCESSING

In [4]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [5]:
def load_labels(file):
  df = pd.read_csv(file, header=None)
  labels = df[0].tolist()
  return labels

def load_data(path, classes, mode="train"): # Return a list a images, a list of bboxes and a list of labels
  images = []
  bboxes = []
  labels = []

  if mode == "train":
    df = pd.read_csv(train_file, header=None)
  else:
    df = pd.read_csv(test_file, header=None)

  for index, row in df.iterrows():
    image_name = row[0]
    x1 = int(row[1])
    y1 = int(row[2])
    x2 = int(row[3])
    y2 = int(row[4])
    label = int(row[5])
    label_name = classes[label-1]

    if mode == "train":
      image = os.path.join(path, "car_data", "car_data", "train", label_name, image_name)
    else:
      image = os.path.join(path, "car_data", "car_data", "test", label_name, image_name)

    bbox = [x1, y1, x2, y2]

    images.append(image)
    bboxes.append(bbox)
    labels.append(label)
  
  return images, bboxes, labels

def load_dataset(path, classes, split=0.2):
  train_images, train_bboxes, train_labels = load_data(path, classes, mode="train")
  train_images, valid_images = train_test_split(train_images, test_size = split, random_state = seed)
  train_bboxes, valid_bboxes = train_test_split(train_bboxes, test_size = split, random_state = seed)
  train_labels, valid_labels = train_test_split(train_labels, test_size = split, random_state = seed)

  test_images, test_bboxes, test_labels = load_data(path, classes, mode="test")

  return (train_images, train_bboxes, train_labels), (valid_images, valid_bboxes, valid_labels), (test_images, test_bboxes, test_labels)

def normalize_image_bbox(path, bbox, label_index):
  # Normalize images [-1,1]
  path = path.decode()
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  if image is None:
    print(path)
    raise ValueError('Image input is None')

  h, w, _ = image.shape
  image = cv2.resize(image, (width, height))
  image = (image - 127.5) / 127.5
  image = image.astype(np.float32)
  
  # Normalize bbox [0, 1]
  x1, y1, x2, y2 = bbox
  norm_x1 = float(x1/w)
  norm_y1 = float(y1/h)
  norm_x2 = float(x2/w)
  norm_y2 = float(y2/h)
  norm_bbox = np.array([norm_x1, norm_y1, norm_x2, norm_y2], dtype=np.float32)

  # One-hot label
  label = [0]*num_classes
  label[label_index - 1] = 1
  one_hot_label = np.array(label, dtype=np.float32)

  return image, norm_bbox, one_hot_label

def preprocess(image, bbox, label):
  image, bbox, label = tf.numpy_function(normalize_image_bbox, [image, bbox, label], [tf.float32, tf.float32, tf.float32])
  image.set_shape((height, width, 3))
  bbox.set_shape(4)
  label.set_shape(num_classes)
  return (image, (bbox, label))

def tf_dataset(images, bboxes, labels, batch_size = batch_size):
  ds = tf.data.Dataset.from_tensor_slices((images, bboxes, labels))
  ds = ds.map(preprocess).batch(batch_size).prefetch(10)
  return ds

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

classes = load_labels(label_file)
(train_images, train_bboxes, train_labels), (val_images, val_bboxes, val_labels), (test_images, test_bboxes, test_label) = load_dataset(os.getcwd(), classes, split = 0.2)

train_ds = tf_dataset(train_images, train_bboxes, train_labels)
val_ds = tf_dataset(val_images, val_bboxes, val_labels)





# BUILD MODEL

In [7]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2


def build_model(input_shape, num_classes = num_classes):
  inputs = layers.Input(input_shape)

  backbone = MobileNetV2(
      include_top = False,
      weights = "imagenet",
      input_tensor = inputs,
      alpha = 1.0
  )
  
  # x = backbone.get_layer("block_14_expand_relu").output
  x = backbone.output
  x = layers.Conv2D(256, 1, padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dropout(0.5)(x)


  # Localization head
  bbox = layers.Dense(4, activation="sigmoid", name = "bbox")(x)
  
  # Classification head
  label = layers.Dense(num_classes, activation="softmax", name="label")(x)

  model = Model(inputs=[inputs], outputs=[bbox, label])
  
  return model


In [8]:
input_shape = (height, width, 3)

mse = tf.keras.losses.MeanSquaredError(
    reduction=tf.keras.losses.Reduction.SUM)

model = build_model(input_shape)
model.compile(loss = {"bbox": mse, "label": "categorical_crossentropy"}, optimizer = Adam(lr))
model.summary()

callbacks = [
        ModelCheckpoint(filepath="/content/drive/MyDrive/GITHUB PROJECTS/Object_localization_classification/model.h5", verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
    ]


9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 160, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 160, 32  128         ['Conv1[0][0]']                  
                               

In [9]:
history = model.fit(train_ds, epochs=num_epochs, validation_data=val_ds)

Epoch 1/100
102/102 [==============================] - 1711s 16s/step - loss: 12.5057 - bbox_loss: 7.0866 - label_loss: 5.4192 - val_loss: 8.0613 - val_bbox_loss: 2.6906 - val_label_loss: 5.3707
Epoch 2/100
102/102 [==============================] - 87s 854ms/step - loss: 6.8017 - bbox_loss: 1.4644 - label_loss: 5.3373 - val_loss: 6.2860 - val_bbox_loss: 0.9986 - val_label_loss: 5.2874
Epoch 3/100
102/102 [==============================] - 87s 845ms/step - loss: 6.0436 - bbox_loss: 0.7821 - label_loss: 5.2614 - val_loss: 5.8566 - val_bbox_loss: 0.6209 - val_label_loss: 5.2356
Epoch 4/100
102/102 [==============================] - 83s 810ms/step - loss: 5.7409 - bbox_loss: 0.5585 - label_loss: 5.1823 - val_loss: 5.6277 - val_bbox_loss: 0.4399 - val_label_loss: 5.1877
Epoch 5/100
102/102 [==============================] - 83s 808ms/step - loss: 5.5646 - bbox_loss: 0.4729 - label_loss: 5.0917 - val_loss: 5.5602 - val_bbox_loss: 0.4327 - val_label_loss: 5.1275
Epoch 6/100
102/102 [========

In [10]:
model.save_weights("./model.h5")
# loaded_model.load_weights("./model.h5")

In [11]:
os.getcwd()

'/content/drive/My Drive/Temp'

# TESTING

In [12]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score

def cal_iou(y_true, y_pred):
    x1 = max(y_true[0], y_pred[0])
    y1 = max(y_true[1], y_pred[1])
    x2 = min(y_true[2], y_pred[2])
    y2 = min(y_true[3], y_pred[3])

    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    true_area = (y_true[2] - y_true[0] + 1) * (y_true[3] - y_true[1] + 1)
    bbox_area = (y_pred[2] - y_pred[0] + 1) * (y_pred[3] - y_pred[1] + 1)

    iou = intersection_area / float(true_area + bbox_area - intersection_area)
    return iou

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

np.random.seed(42)
tf.random.set_seed(42)

create_dir("./results")

# dataset_path = os.getcwd()()

# model_path = os.path.join("files", "model.h5")

# model = tf.keras.models.load_model(model_path)



test_images = test_images[:1000]
test_bboxes = test_bboxes[:1000]
test_labels = test_label[:1000]
print(f"Test : {len(test_images)} - {len(test_bboxes)} - {len(test_label)}")

mean_iou = []
pred_labels = []

for image, true_bbox, true_labels in tqdm(zip(test_images, test_bboxes, test_label), total=len(test_images)):
        name = image.split("/")[-1]
        image = cv2.imread(image, cv2.IMREAD_COLOR)
        
        x = cv2.resize(image, (width, height))
        print(name)
        x = (x - 127.5) / 127.5
        x = np.expand_dims(x, axis=0)

        true_x1, true_y1, true_x2, true_y2 = true_bbox

        pred_bbox, label = model.predict(x, verbose=0)
        pred_bbox = pred_bbox[0]
        label_index = np.argmax(label[0])
        pred_labels.append(label_index+1)

        pred_x1 = int(pred_bbox[0] * image.shape[1])
        pred_y1 = int(pred_bbox[1] * image.shape[0])
        pred_x2 = int(pred_bbox[2] * image.shape[1])
        pred_y2 = int(pred_bbox[3] * image.shape[0])

        iou = cal_iou(true_bbox, [pred_x1, pred_y1, pred_x2, pred_y2])
        mean_iou.append(iou)

        image = cv2.rectangle(image, (true_x1, true_y1), (true_x2, true_y2), (255, 0, 0), 2) ## BLUE
        image = cv2.rectangle(image, (pred_x1, pred_y1), (pred_x2, pred_y2), (0, 0, 255), 2) ## RED

        font_size = 1
        pred_class_name = classes[label_index]
        text = f"{pred_class_name}"
        cv2.putText(image, text, (pred_x1, pred_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 255), 2)

        font_size = 1
        true_class_name = classes[true_labels-1]
        text = f"{true_class_name}"
        cv2.putText(image, text, (true_x1, true_y1-10), cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 0, 0), 2)

        x = 50
        y = 50
        font_size = 1
        cv2.putText(image, f"IoU: {iou:.4f}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 0, 0), 2)
        # print(f"Image Name: {name} - IoU: {iou:.4f} - Pred label: {pred_class_name} - True label: {true_class_name}")

        cv2.imwrite(f"results/{name}", image)

score = np.mean(mean_iou, axis=0)
mean_acc = accuracy_score(test_labels, pred_labels)
print(f"Mean IoU: {score:.4f} - Acc: {mean_acc:.4f}")

Test : 1000 - 1000 - 8041


  0%|          | 0/1000 [00:00<?, ?it/s]

00001.jpg


  0%|          | 2/1000 [00:03<27:18,  1.64s/it]

00002.jpg


...

00998.jpg


100%|█████████▉| 999/1000 [07:03<00:00,  2.11it/s]

00999.jpg


100%|██████████| 1000/1000 [07:03<00:00,  2.36it/s]

01000.jpg
Mean IoU: 0.8542 - Acc: 0.6840
